### Importing useful libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

### Loading Dataset

In [2]:
#url = 'https://raw.githubusercontent.com/WalePhenomenon/climate_change/master/fuel_ferc1.csv'
#fuel_data = pd.read_csv(url)
fuel_data = pd.read_csv('./Documents/MyHamoyeProjects/stageA/Dataset.csv')
fuel_data = fuel_data.drop('Unnamed: 0', axis=1) # dropping the 'unnamed: 0' column created probably from the index of the data
fuel_data.head()

,record_id,utility_id_ferc1,report_year,plant_name_ferc1,fuel_type_code_pudl,fuel_unit,fuel_qty_burned,fuel_mmbtu_per_unit,fuel_cost_per_unit_burned,fuel_cost_per_unit_delivered,fuel_cost_per_mmbtu
0,f1_fuel_1994_12_1_0_7,1,1994,rockport,coal,ton,5377489.0,16.590,18.59,18.53,1.121
1,f1_fuel_1994_12_1_0_10,1,1994,rockport total plant,coal,ton,10486945.0,16.592,18.58,18.53,1.120
2,f1_fuel_1994_12_2_0_1,2,1994,gorgas,coal,ton,2978683.0,24.130,39.72,38.12,1.650
3,f1_fuel_1994_12_2_0_7,2,1994,barry,coal,ton,3739484.0,23.950,47.21,45.99,1.970
4,f1_fuel_1994_12_2_0_10,2,1994,chickasaw,gas,mcf,40533.0,1.000,2.77,2.77,2.570


In [3]:
fuel_data.isna().sum() # checking for missing values

record_id                         0
utility_id_ferc1                  0
report_year                       0
plant_name_ferc1                  0
fuel_type_code_pudl               0
fuel_unit                       180
fuel_qty_burned                   0
fuel_mmbtu_per_unit               0
fuel_cost_per_unit_burned         0
fuel_cost_per_unit_delivered      0
fuel_cost_per_mmbtu               0
dtype: int64

In [4]:
fuel_data.describe() # checking summary statistics

,utility_id_ferc1,report_year,fuel_qty_burned,fuel_mmbtu_per_unit,fuel_cost_per_unit_burned,fuel_cost_per_unit_delivered,fuel_cost_per_mmbtu
count,29523.000000,29523.000000,2.952300e+04,29523.000000,29523.000000,2.952300e+04,29523.000000
mean,118.601836,2005.806050,2.622119e+06,8.492111,208.649031,9.175704e+02,19.304354
std,74.178353,7.025483,9.118004e+06,10.600220,2854.490090,6.877593e+04,2091.540939
min,1.000000,1994.000000,1.000000e+00,0.000001,-276.080000,-8.749370e+02,-41.501000
25%,55.000000,2000.000000,1.381700e+04,1.024000,5.207000,3.778500e+00,1.940000
50%,122.000000,2006.000000,2.533220e+05,5.762694,26.000000,1.737100e+01,4.127000
75%,176.000000,2012.000000,1.424034e+06,17.006000,47.113000,4.213700e+01,7.745000
max,514.000000,2018.000000,5.558942e+08,341.260000,139358.000000,7.964521e+06,359278.000000


In [5]:
# retrieving the mode of the feature with missing values to later use for imputation

fuel_units_count = fuel_data.groupby('fuel_unit').count()['record_id'] 
fuel_units_mode = fuel_units_count.sort_values(ascending=False).index[0]

In [6]:
# imputating the features with missing values
fuel_data_imputated = fuel_data.fillna(fuel_units_mode)
fuel_data_imputated.isna().sum()

record_id                       0
utility_id_ferc1                0
report_year                     0
plant_name_ferc1                0
fuel_type_code_pudl             0
fuel_unit                       0
fuel_qty_burned                 0
fuel_mmbtu_per_unit             0
fuel_cost_per_unit_burned       0
fuel_cost_per_unit_delivered    0
fuel_cost_per_mmbtu             0
dtype: int64

### Question 1

In [7]:
A = [1, 2, 3, 4, 5, 6]
B = [13, 21, 34]
A_B = A + B
A_B

[1, 2, 3, 4, 5, 6, 13, 21, 34]

##### A.append(B) and A.extend(B) will only mutate A but A_B will be None

### Question 2

In [8]:
np.identity(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

### Question 3

In [9]:
mean_fcpub_series = fuel_data_imputated.groupby('fuel_type_code_pudl').mean()['fuel_cost_per_unit_burned']
mean_fcpub_series.sort_values().index[0]

'gas'

### Question 4

In [10]:
fuel_data_imputated['fuel_mmbtu_per_unit'].std(ddof=0) # ddof=0 so as to calculate standard deviation of population 
                                                        # and not that of sample

10.600040781504145

In [11]:
fuel_data_imputated['fuel_mmbtu_per_unit'].quantile(q=0.75)

17.006

### Question 5

In [12]:
fuel_data_imputated['fuel_qty_burned'].skew()

15.851495469109503

In [13]:
fuel_data_imputated['fuel_qty_burned'].kurt()

651.3694501337732

### Question 6

In [14]:
missing_feat_series = fuel_data.isnull().sum()
missing_feat_series[missing_feat_series != 0]

fuel_unit    180
dtype: int64

In [15]:
percentage_missing = (missing_feat_series['fuel_unit'] / fuel_data['fuel_unit'].shape[0]) * 100
percentage_missing

0.609694136774718

### Question 7

In [16]:
fuel_data['fuel_unit'].unique()

array(['ton', 'mcf', 'kgU', 'bbl', 'gramsU', nan, 'mwdth', 'mmbtu',
       'mwhth', 'gal'], dtype=object)

 ##### As seen above, the values of 'fuel_unit' are categorical, meaning mode imputation will be the best option.

### Question 8

In [17]:
feat_corr = fuel_data_imputated.corr().loc['fuel_cost_per_unit_burned']
feat_corr.sort_values().index[1:3].to_list()

['fuel_qty_burned', 'fuel_mmbtu_per_unit']

### Question 9

In [18]:
coal_df = fuel_data_imputated[fuel_data_imputated['fuel_type_code_pudl'] == 'coal']
coal94 = coal_df[coal_df['report_year'] == 1994]['fuel_cost_per_unit_burned'].sum()
coal98 = coal_df[coal_df['report_year'] == 1998]['fuel_cost_per_unit_burned'].sum()

percentage_change = ((coal98 - coal94) / coal94) * 100
percentage_change

-20.567654518260518

##### The closest answer in the option is -21%. Since the question didn't ask us to approximate; I chose 'None of the above'.

### Question 10

In [19]:
yearly_series = fuel_data_imputated.groupby('report_year').mean()['fuel_cost_per_unit_delivered']
yearly_series.sort_values(ascending=False).index[0]

1997